In [144]:
#Read the file 

In [145]:
with open('Data/input.txt', 'r') as f:

    text = f.read()

In [146]:
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [147]:
#process the text , rmove url and some charcters

In [148]:
import re

def preprocess_text(text):
    
    if not isinstance(text, str):
        return ""

    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    text = re.sub(r"/[^\s]+", "", text)

    text = re.sub(r'\n', '', text)

    return text.strip()

In [149]:
text = preprocess_text(text)

In [150]:
text[:100]

'First Citizen:Before we proceed any further, hear me speak.All:Speak, speak.First Citizen:You are al'

In [151]:
print('Lenght of my text', len(text))

Lenght of my text 1075394


In [152]:
chars = sorted(list(set(text)))

vocab = len(chars)

print(''.join(chars))
print(vocab)

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
64


In [153]:
# Tokeization in the character level

In [154]:
ztoa = {i:k for i,k in enumerate(chars)}
atoz = {k:i for i,k in enumerate(chars)}

encoding = lambda word: [atoz[w] for w in word]
decoding = lambda word: "".join([ztoa[w] for w in word])

print(encoding('holla hye'))
print(decoding(encoding('holla hye')))

[45, 52, 49, 49, 38, 0, 45, 62, 42]
holla hye


In [155]:
import torch

encoding_text = torch.tensor(encoding(text), dtype= torch.long)

print(encoding_text.shape, encoding_text.dtype)
print(encoding_text[: 100])

torch.Size([1075394]) torch.int64
tensor([17, 46, 55, 56, 57,  0, 14, 46, 57, 46, 63, 42, 51,  9, 13, 42, 43, 52,
        55, 42,  0, 60, 42,  0, 53, 55, 52, 40, 42, 42, 41,  0, 38, 51, 62,  0,
        43, 58, 55, 57, 45, 42, 55,  5,  0, 45, 42, 38, 55,  0, 50, 42,  0, 56,
        53, 42, 38, 48,  7, 12, 49, 49,  9, 30, 53, 42, 38, 48,  5,  0, 56, 53,
        42, 38, 48,  7, 17, 46, 55, 56, 57,  0, 14, 46, 57, 46, 63, 42, 51,  9,
        36, 52, 58,  0, 38, 55, 42,  0, 38, 49])


In [156]:
#split train validation

In [157]:
n = int(0.9*len(encoding_text))

train_data = encoding_text[:n]
valid_data = encoding_text[n:]

In [158]:
# Training with all the data at once is costly, but we can train it with chuncks

In [159]:
train_data[: 8]

tensor([17, 46, 55, 56, 57,  0, 14, 46])

In [160]:
train_data[1: 9]

tensor([46, 55, 56, 57,  0, 14, 46, 57])

In [161]:
block_size = 8

x = train_data[: block_size]

y = train_data[1: block_size + 1]

for i in range(len(train_data[: block_size])):

    o = x[:i+1]
    j = y[i]

    print(f'whne input is {o} target is {j}')

whne input is tensor([17]) target is 46
whne input is tensor([17, 46]) target is 55
whne input is tensor([17, 46, 55]) target is 56
whne input is tensor([17, 46, 55, 56]) target is 57
whne input is tensor([17, 46, 55, 56, 57]) target is 0
whne input is tensor([17, 46, 55, 56, 57,  0]) target is 14
whne input is tensor([17, 46, 55, 56, 57,  0, 14]) target is 46
whne input is tensor([17, 46, 55, 56, 57,  0, 14, 46]) target is 57


In [162]:
# Random sampling

In [163]:
batch_size = 4

def random_sampling(split):

    data = train_data if split == 'train' else valid_data

    ix = torch.randint(len(data) - block_size, (batch_size,))

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x, y

x, y = random_sampling('train')

In [164]:
x[0][:2]

tensor([38, 44])

In [165]:
x

tensor([[38, 44, 42, 50, 42, 51, 57,  9],
        [62,  0, 39, 42,  0, 38,  0, 53],
        [60, 11, 27, 55, 52, 59, 52, 56],
        [52, 55, 51,  0, 45, 46, 56,  0]])

In [166]:
y

tensor([[44, 42, 50, 42, 51, 57,  9, 31],
        [ 0, 39, 42,  0, 38,  0, 53, 55],
        [11, 27, 55, 52, 59, 52, 56, 57],
        [55, 51,  0, 45, 46, 56,  0, 40]])

In [167]:
for i in range(4):

    for j in range(8):

        x_l = x[i][:j+1]
        y_l = y[i][j]

        print(f"when input is {x_l} the target is {y_l}")



when input is tensor([38]) the target is 44
when input is tensor([38, 44]) the target is 42
when input is tensor([38, 44, 42]) the target is 50
when input is tensor([38, 44, 42, 50]) the target is 42
when input is tensor([38, 44, 42, 50, 42]) the target is 51
when input is tensor([38, 44, 42, 50, 42, 51]) the target is 57
when input is tensor([38, 44, 42, 50, 42, 51, 57]) the target is 9
when input is tensor([38, 44, 42, 50, 42, 51, 57,  9]) the target is 31
when input is tensor([62]) the target is 0
when input is tensor([62,  0]) the target is 39
when input is tensor([62,  0, 39]) the target is 42
when input is tensor([62,  0, 39, 42]) the target is 0
when input is tensor([62,  0, 39, 42,  0]) the target is 38
when input is tensor([62,  0, 39, 42,  0, 38]) the target is 0
when input is tensor([62,  0, 39, 42,  0, 38,  0]) the target is 53
when input is tensor([62,  0, 39, 42,  0, 38,  0, 53]) the target is 55
when input is tensor([60]) the target is 11
when input is tensor([60, 11]) t